# (CTC-17) Projeto de Buscas
---

## Parâmetros Editáveis da Busca

In [94]:
# Nesta célula, há 3 variáveis que armazenam os parâmetros da busca (conforme especificado no roteiro):
#   file_name   => nome do arquivo de dados
#   source      => cidade de origem na busca
#   destination => cidade de destino na busca

# string com o nome (endereço) do arquivo de dados.
file_name = 'australia.csv'

# source e destination => podem ser string ou int

# Nome das cidades (string)
source = 'Alice Springs'
destination = 'Yulara'

# Id das cidades (int)
# source = 5
# destination = 219


## Dependências

In [95]:
import random
import pandas as pd
import heapq as heap
from math import sqrt
from tqdm import tqdm

## Importando os dados

In [96]:
# Import
australia = pd.read_csv(file_name, sep = ",")
# Cleaning
australia = australia[['id', 'city', 'lat', 'lng']]
# Printing
australia

,id,city,lat,lng
0,1,Adelaide,-34.928661,138.598633
1,2,Adelaide River,-13.239430,131.107330
2,3,Albany,-35.003101,117.865952
3,4,Albury,-36.074823,146.924006
4,5,Alice Springs,-23.697479,133.883621
...,...,...,...,...
214,215,Woomera,-31.199810,136.832581
215,216,Yamba,-29.435450,153.360306
216,217,Yeppoon,-23.126829,150.744064
217,218,Young,-34.313499,148.301071


## Pré-Game

### Estrutura de Dados

In [97]:
class City:
    def __init__(self, id, name, lat, lng):
        self.id = id
        self.name = name
        self.lat = lat
        self.lng = lng
        self.roads = set()
        self.weight = None

    def successors(self):
        for road in self.roads:
            yield road.city, road.length

    def __str__(self):
        return '<City id={} name=\'{}\' weight={:g} pos=({:g}, {:g}) #roads={}>'.format(
            self.id, self.name, self.weight, self.lat,      self.lng, len(self.roads)
        )
    
        
class Road:
    def __init__(self, city, length):
        self.city = city
        self.length = length
        
def add_road_between(city1, city2):
    # 1.1 => a distância real é 10% maior
    distance = 1.1 * sqrt((city1.lat - city2.lat)**2 + (city1.lng - city2.lng)**2)
    # city1 -> city2 (relação é simétrica)
    road1 = Road(city2, distance)
    city1.roads.add(road1)
    # city2 -> city1 (relação é simétrica)
    road2 = Road(city1, distance)
    city2.roads.add(road2)

### Preenchendo a estrutura de dados

In [98]:
cities = [None] # None é só para preencher o id = 0 (outra alternativa seria trabalhar com `índice - 1`)

# Cities
for index, row in australia.iterrows():
    city = City(row['id'], row['city'], row['lat'], row['lng'])
    cities.append(city)
    
# Roads
for city in cities:
    if city:
        if city.id % 2 == 0:
            # Road to (x - 1)
            add_road_between(city, cities[city.id - 1])
            
            # Road to (x + 2) if it exists
            if city.id + 2 < len(cities):
                add_road_between(city, cities[city.id + 2])
                
        elif city.id > 2:
            # Road to (x - 2)
            add_road_between(city, cities[city.id - 2])
            
            # Road to (x + 1) if it exists
            if city.id + 1 < len(cities):
                add_road_between(city, cities[city.id + 1])

## Algoritmo de Busca

### Adiciona informação (Busca Informada)

In [99]:
# Calcula as distâncias das cidades até o destino (em linha reta) 
# Esta será a heurística para o algoritmo A*

# Encontra o nó da cidade destino
if type(destination) == str:
    for city in cities:
        if city and city.name == destination:
            dest_city = city
            break
        dest_city = None
else:
    for city in cities:
        if city and city.id == destination:
            dest_city = city
            break
        dest_city = None
        

# Encontra o nó da cidade fonte
if type(source) == str:
    for city in cities:
        if city and city.name == source:
            src_city = city
            break
        src_city = None
else:
    for city in cities:
        if city and city.id == source:
            src_city = city
            break
        src_city = None

if dest_city == None or src_city == None:
    print('Cidades não encontradas')


# Para cada distância seta o peso como a distância em linha reta até a cidade destino
for city in cities:
    if city:
        city.weight = sqrt((city.lat - dest_city.lat)**2 + (city.lng - dest_city.lng)**2)

In [100]:
# Checando as cidades de origem e de destino
print('Cidade de Origem:  ', src_city)
print('Cidade de Destino: ', dest_city)

# Checando 5 cidades aleatoriamente
print('\nConferindo 5 exemplos de cidade:')
for city in random.sample(cities, 5):
    print(city)

Cidade de Origem:   &lt;City id=5 name=&#39;Alice Springs&#39; weight=3.28992 pos=(-23.6975, 133.884) #roads=4&gt;
Cidade de Destino:  &lt;City id=219 name=&#39;Yulara&#39; weight=0 pos=(-25.2453, 130.981) #roads=1&gt;

Conferindo 5 exemplos de cidade:
&lt;City id=219 name=&#39;Yulara&#39; weight=0 pos=(-25.2453, 130.981) #roads=1&gt;
&lt;City id=36 name=&#39;Byron Bay&#39; weight=22.8849 pos=(-28.642, 153.612) #roads=4&gt;
&lt;City id=123 name=&#39;Mildura&#39; weight=14.3166 pos=(-34.1855, 142.163) #roads=4&gt;
&lt;City id=138 name=&#39;Norseman&#39; weight=11.533 pos=(-32.1977, 121.779) #roads=4&gt;
&lt;City id=130 name=&#39;Mount Magnet&#39; weight=13.4308 pos=(-28.0648, 117.849) #roads=4&gt;


### Algoritmo A*

In [101]:
# Implementação do algoritmo A*, baseado no pseudo-código e descrições apresentadas nas aulas de teoria
# Notação: f(n) = g(n) + h(n)
#   f(n) => função de avaliação
#   g(n) => custo (real) da trajetória até nó n (inclusive)
#   h(n) => heurística do custo até o estado objetivo (atributo weight de City)

# Essa classe representa um nó expandido na busca.
# Há uma referência ao nó pai (que o expandiu), assim é possível reconstruir a trajetória.
# O operador de comparação é necessário para a fila de prioridades (usamos f_value na comparação).
class Node:
    def __init__(self, f_value, g_value, element, parent):
        self.f_value = f_value
        self.g_value = g_value
        self.element = element
        self.parent = parent

    def __lt__(self, other):
        if self.f_value != other.f_value:
            return self.f_value < other.f_value
        else:
            return self.element.id < other.element.id

# Função que efetua a busca usando o algoritmo A*
# initial_state: estado inicial (cidade de origem)
# goal_test: função que checa se o estado é o objetivo
def astar_search(initial_state, goal_test, max_iterations = 1000):
    # Para diminuir o número de nós expandidos, salvamos as cidades já visitadas
    expanded = set()

    # Inicializando a heap de minimo
    pq = [Node(initial_state.weight, 0, initial_state, None)]
    heap.heapify(pq)

    for _ in tqdm(range(max_iterations)):
        # não há caminho
        if len(pq) == 0:
            return None

        # extraindo o nó de menor avaliação para expansão
        curr = heap.heappop(pq)

        # verificando se a cidade já foi expandida
        if curr.element in expanded:
            continue
        else:
            expanded.add(curr.element)

        # encontrou o objetivo?
        if goal_test(curr.element):
            return curr

        # expandindo o nó (iterando sobre todos os sucessores da cidade)
        # price é o custo da transição
        for succ, price in curr.element.successors():
            # evitando adicionar à heap cidades já expandidas
            if succ in expanded:
                continue

            # computando f(n') e g(n')
            new_g_value = (curr.g_value + price)
            new_f_value = new_g_value + succ.weight

            # inserindo novo nó na heap
            heap.heappush(pq, Node(new_f_value, new_g_value, succ, curr))

In [102]:
def goal_test(city):
    return city == dest_city

solution_node = astar_search(src_city, goal_test)

print('\n\nCidade Final: ', solution_node.element)

 43%|████▎     | 432/1000 [00:00&lt;00:00, 265058.42it/s]

Cidade Final:  &lt;City id=219 name=&#39;Yulara&#39; weight=0 pos=(-25.2453, 130.981) #roads=1&gt;



## Resultado

In [103]:
# extraindo a trajetória percorrida 
path = []

# percorrendo a lista encadeada de nós, a partir do destino
# até a origem, por meio do campo parent de Node.
aux_node = solution_node
while aux_node is not None:
    path.append(aux_node.element)
    aux_node = aux_node.parent
path.reverse()

# Resultados da solução obtida:

print('LENGTH: {}'.format(len(path)))
print('COST: {:g}'.format(solution_node.f_value))
print('TRAJECTORY:')

print('{:3d}) [START] {}'.format(0, path[0].name))
for i in range(len(path)):
    if i == len(path) - 1:
        print('{:3d}) {} [THE END]'.format(i+1, path[i].name))
    else:
        print('{:3d}) {} -> {}'.format(i+1, path[i].name, path[i+1].name))

LENGTH: 124
COST: 1647.34
TRAJECTORY:
  0) [START] Alice Springs
  1) Alice Springs -&gt; Andamooka
  2) Andamooka -&gt; Armidale
  3) Armidale -&gt; Ayr
  4) Ayr -&gt; Ballarat
  5) Ballarat -&gt; Bairnsdale East
  6) Bairnsdale East -&gt; Ballina
  7) Ballina -&gt; Batemans Bay
  8) Batemans Bay -&gt; Bathurst
  9) Bathurst -&gt; Bendigo
 10) Bendigo -&gt; Bicheno
 11) Bicheno -&gt; Birdsville
 12) Birdsville -&gt; Bordertown
 13) Bordertown -&gt; Bourke
 14) Bourke -&gt; Brisbane
 15) Brisbane -&gt; Broome
 16) Broome -&gt; Bundaberg
 17) Bundaberg -&gt; Burnie
 18) Burnie -&gt; Byron Bay
 19) Byron Bay -&gt; Cairns
 20) Cairns -&gt; Caboolture
 21) Caboolture -&gt; Caloundra
 22) Caloundra -&gt; Canberra
 23) Canberra -&gt; Ceduna
 24) Ceduna -&gt; Charleville
 25) Charleville -&gt; Clare
 26) Clare -&gt; Cobram
 27) Cobram -&gt; Colac
 28) Colac -&gt; Cowell
 29) Cowell -&gt; Cranbourne
 30) Cranbourne -&gt; Dalby
 31) Dalby -&gt; Deniliquin
 32) Deniliquin -&gt; Dubbo
 33) Dubbo 

## Testes

### Cidades Romenas

In [104]:
def add_road_dist(city1, city2, dist):
    city1.roads.add(Road(city2, dist))
    city2.roads.add(Road(city1, dist))

# Cidades

cities = []

c1 = City(1, 'Oradea', 0, 0)
c1.weight = 380
cities.append(c1)

c2 = City(2, 'Zerind', 0, 0)
c2.weight = 374
cities.append(c2)

c3 = City(3, 'Arad', 0, 0)
c3.weight = 366
cities.append(c3)

c4 = City(4, 'Sibiu', 0, 0)
c4.weight = 253
cities.append(c4)

c5 = City(5, 'Fagaras', 0, 0)
c5.weight = 178
cities.append(c5)

c6 = City(6, 'Timisoara', 0, 0)
c6.weight = 329
cities.append(c6)

c7 = City(7, 'Lugoj', 0, 0)
c7.weight = 244
cities.append(c7)

c8 = City(8, 'Rimnicu Vilcea', 0, 0)
c8.weight = 193
cities.append(c8)

c9 = City(9, 'Mehadia', 0, 0)
c9.weight = 241
cities.append(c9)

c10 = City(10, 'Pitesti', 0, 0)
c10.weight = 98
cities.append(c10)

c11 = City(11, 'Dobreta', 0, 0)
c11.weight = 242
cities.append(c11)

c12 = City(12, 'Craiova', 0, 0)
c12.weight = 160
cities.append(c12)

c13 = City(13, 'Bucharest', 0, 0)
c13.weight = 0
cities.append(c13)

c14 = City(14, 'Giurgiu', 0, 0)
c14.weight = 77
cities.append(c14)

c15 = City(15, 'Urziceni', 0, 0)
c15.weight = 80
cities.append(c15)

c16 = City(16, 'Hirsova', 0, 0)
c16.weight = 151
cities.append(c16)

c17 = City(17, 'Eforie', 0, 0)
c17.weight = 161
cities.append(c17)

c18 = City(18, 'Vaslui', 0, 0)
c18.weight = 199
cities.append(c18)

c19 = City(19, 'Iasi', 0, 0)
c19.weight = 226
cities.append(c19)

c20 = City(20, 'Neamt', 0, 0)
c20.weight = 234
cities.append(c20)

# Estradas

add_road_dist(c1, c2, 71)
add_road_dist(c1, c4, 151)
add_road_dist(c2, c3, 75)
add_road_dist(c3, c4, 140)

add_road_dist(c3, c6, 118)
add_road_dist(c4, c5, 99)
add_road_dist(c4, c6, 80)
add_road_dist(c6, c7, 111)

add_road_dist(c7, c9, 70)
add_road_dist(c9, c11, 75)
add_road_dist(c11, c12, 120)

add_road_dist(c8, c12, 146)
add_road_dist(c8, c10, 97)
add_road_dist(c12, c10, 138)

add_road_dist(c5, c13, 211)
add_road_dist(c10, c13, 101)
add_road_dist(c14, c13, 90)
add_road_dist(c13, c15, 85)

add_road_dist(c15, c16, 98)
add_road_dist(c16, c17, 86)

add_road_dist(c13, c18, 142)
add_road_dist(c18, c19, 92)
add_road_dist(c19, c20, 87)

result = astar_search(c3, lambda city: city == c13)
print('\n\nCidade Final: ', result.element)
print('Preco: ', result.f_value, '\n')

aux_node = result
while aux_node is not None:
    print(aux_node.element)
    aux_node = aux_node.parent


  0%|          | 5/1000 [00:00&lt;00:00, 32214.32it/s]

Cidade Final:  &lt;City id=13 name=&#39;Bucharest&#39; weight=0 pos=(0, 0) #roads=5&gt;
Preco:  450 

&lt;City id=13 name=&#39;Bucharest&#39; weight=0 pos=(0, 0) #roads=5&gt;
&lt;City id=5 name=&#39;Fagaras&#39; weight=178 pos=(0, 0) #roads=2&gt;
&lt;City id=4 name=&#39;Sibiu&#39; weight=253 pos=(0, 0) #roads=4&gt;
&lt;City id=3 name=&#39;Arad&#39; weight=366 pos=(0, 0) #roads=3&gt;

